<a href="https://colab.research.google.com/github/jyrj/deeplearning-masterclass-tensorflow/blob/master/Transfer_Learning_and_Fine_Tunning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Downloading the Dogs vs Cats dataset 

In [0]:
!pip install tensorflow-gpu==2.0

     |████████████████████████████████| 380.8MB 42kB/s 
     |████████████████████████████████| 450kB 50.9MB/s 
     |████████████████████████████████| 3.8MB 55.5MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=7f279ae6e440f3c95a76814b5465c851b45bf89f0e285d0765777c420e74a6ad
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow 2.2.0 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: tensorflow 2.2.0 has requirement tensorboard<2.3.0,>=2.2.0, but you'll have tensorboard 2.0.2 which is incompatible.
ERROR: tensorflow 2.2.0 has requirement tensorflow-estimator<2.3.0,>=2.2.0, but you'll have tensorflow-estimator 2.0.1 which is incompatible.
ERROR: tensorflow-probability 0.10.0rc0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorflow-estimator 2.2.0
    Unins

In [0]:
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O ./cats_and_dogs_filtered.zip

--2020-05-13 09:56:52--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.195.128, 2607:f8b0:400e:c07::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.195.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘./cats_and_dogs_filtered.zip’

./cats_and_dogs_fil 100%[===================>]  65.43M   121MB/s    in 0.5s    

2020-05-13 09:56:53 (121 MB/s) - ‘./cats_and_dogs_filtered.zip’ saved [68606236/68606236]



## Stage 2: Dataset preprocessing

### Import dependencies

In [0]:
import os
import zipfile
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook
from tensorflow.keras.preprocessing.image import ImageDataGenerator

%matplotlib inline
tf.__version__

'2.0.0'

### Unzipping the Dogs vs Cats dataset

In [0]:
dataset_path = "./cats_and_dogs_filtered.zip"

In [0]:
zip_object = zipfile.ZipFile(file=dataset_path, mode="r")

In [0]:
zip_object.extractall("./")

In [0]:
zip_object.close()

### Seting up dataset paths

In [0]:
dataset_path_new = "./cats_and_dogs_filtered/"

In [0]:
train_dir = os.path.join(dataset_path_new, "train")
validation_dir = os.path.join(dataset_path_new, "validation")

## Building the model

### Loading the pre-trained model (MobileNetV2)

In [0]:
IMG_SHAPE = (128, 128, 3)

In [0]:
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights="imagenet")

9412608/9406464 [==============================] - 1s 0us/step


In [0]:
base_model.summary()

Model: "mobilenetv2_1.00_128"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
_______________________________________________________________________________

### Freezing the base model

In [0]:
base_model.trainable = False

### Defining the custom head for our network

In [0]:
base_model.output

<tf.Tensor 'out_relu/Identity:0' shape=(None, 4, 4, 1280) dtype=float32>

In [0]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)

In [0]:
global_average_layer

<tf.Tensor 'global_average_pooling2d/Identity:0' shape=(None, 1280) dtype=float32>

In [0]:
dense=tf.keras.layers.Dense(units=1000,activation='relu')(global_average_layer)
dropout=tf.keras.layers.Dropout(0.3)(dense)

In [0]:
prediction_layer = tf.keras.layers.Dense(units=1, activation='sigmoid')(dropout)

### Defining the model

In [0]:
model = tf.keras.models.Model(inputs=base_model.input, outputs=prediction_layer)

In [0]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
______________________________________________________________________________________________

### Compiling the model

In [0]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001), loss="binary_crossentropy", metrics=["accuracy"])

### Creating Data Generators

Resizing images

    Big pre-trained architecture support only certain input sizes.

 MobileNet (architecture that we use) supports: (96, 96), (128, 128), (160, 160), (192, 192), (224, 224).

In [0]:
data_gen_train = ImageDataGenerator(rescale=1/255.)
data_gen_valid = ImageDataGenerator(rescale=1/255.)

In [0]:
train_generator = data_gen_train.flow_from_directory(train_dir, target_size=(128,128), batch_size=40, class_mode="binary")

Found 2000 images belonging to 2 classes.


In [0]:
valid_generator = data_gen_valid.flow_from_directory(validation_dir, target_size=(128,128), batch_size=40, class_mode="binary")

Found 1000 images belonging to 2 classes.


### Training the model

In [0]:
model.fit_generator(train_generator, epochs=10, validation_data=valid_generator)

Epoch 1/10
50/50 [==============================] - 19s 377ms/step - loss: 0.3080 - accuracy: 0.8670 - val_loss: 0.3398 - val_accuracy: 0.8440
Epoch 2/10
50/50 [==============================] - 13s 266ms/step - loss: 0.1742 - accuracy: 0.9285 - val_loss: 0.1176 - val_accuracy: 0.9540
Epoch 3/10
50/50 [==============================] - 13s 263ms/step - loss: 0.1422 - accuracy: 0.9455 - val_loss: 0.1757 - val_accuracy: 0.9370
Epoch 4/10
50/50 [==============================] - 13s 262ms/step - loss: 0.1096 - accuracy: 0.9590 - val_loss: 0.1230 - val_accuracy: 0.9570
Epoch 5/10
50/50 [==============================] - 13s 261ms/step - loss: 0.1082 - accuracy: 0.9640 - val_loss: 0.1490 - val_accuracy: 0.9470
Epoch 6/10
50/50 [==============================] - 13s 269ms/step - loss: 0.0916 - accuracy: 0.9650 - val_loss: 0.2013 - val_accuracy: 0.9330
Epoch 7/10
50/50 [==============================] - 13s 264ms/step - loss: 0.0932 - accuracy: 0.9685 - val_loss: 0.1533 - val_accuracy: 0.9510

### Transfer learning model evaluation

In [0]:
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)

In [0]:
print("Accuracy after transfer learning: {}".format(valid_accuracy))

Accuracy after transfer learning: 0.9580000042915344


## Fine tuning


There are a few pointers:

- DO NOT use Fine tuning on the whole network; only a few top layers are enough. In most cases, they are more specialized. The goal of the Fine-tuning is to adopt that specific part of the network for our custom (new) dataset.
- Start with the fine tunning AFTER you have finished with transfer learning step. If we try to perform Fine tuning immediately, gradients will be much different between our custom head layer and a few unfrozen layers from the base model. 

### Un-freeze a few top layers from the model

In [0]:
base_model.trainable = True

In [0]:
print("Number of layersin the base model: {}".format(len(base_model.layers)))

Number of layersin the base model: 155


In [0]:
fine_tune_at = 90

In [0]:
for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

### Compiling the model for fine-tuning

In [0]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=0.0001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

### Fine tuning

In [0]:
model.fit_generator(train_generator,  
                    epochs=10, 
                    validation_data=valid_generator)

Epoch 1/10
50/50 [==============================] - 16s 326ms/step - loss: 0.0987 - accuracy: 0.9640 - val_loss: 0.1540 - val_accuracy: 0.9550
Epoch 2/10
50/50 [==============================] - 15s 308ms/step - loss: 0.0180 - accuracy: 0.9960 - val_loss: 0.1628 - val_accuracy: 0.9590
Epoch 3/10
50/50 [==============================] - 16s 311ms/step - loss: 0.0300 - accuracy: 0.9905 - val_loss: 0.4173 - val_accuracy: 0.9090
Epoch 4/10
50/50 [==============================] - 15s 307ms/step - loss: 0.0115 - accuracy: 0.9965 - val_loss: 0.3952 - val_accuracy: 0.9230
Epoch 5/10
50/50 [==============================] - 16s 310ms/step - loss: 0.0156 - accuracy: 0.9975 - val_loss: 0.4689 - val_accuracy: 0.9140
Epoch 6/10
50/50 [==============================] - 15s 308ms/step - loss: 0.0071 - accuracy: 0.9970 - val_loss: 0.3348 - val_accuracy: 0.9340
Epoch 7/10
50/50 [==============================] - 15s 308ms/step - loss: 0.0145 - accuracy: 0.9955 - val_loss: 0.9279 - val_accuracy: 0.8540

### Evaluating the fine tuned model

In [0]:
valid_loss, valid_accuracy = model.evaluate_generator(valid_generator)

In [0]:
print("Validation accuracy after fine tuning: {}".format(valid_accuracy))

Validation accuracy after fine tuning: 0.9300000071525574
